<a href="https://colab.research.google.com/github/Abbujaa/Abbujaa.github.io/blob/main/1-1_Pipelines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lesson 1.1: ML Pipelines with ZenML

[![Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/zenml-io/zenbytes/blob/main/1-1_Pipelines.ipynb)

***Key Concepts:*** *ML Pipelines, Steps*

In this notebook, we will learn how to easily convert existing ML code into ML pipelines using ZenML.

Since we will build models with [sklearn](https://scikit-learn.org/stable/), you will need to have the ZenML sklearn integration installed. You can install ZenML and the sklearn integration with the following command, which will also restart the kernel of your notebook.

In [1]:
%pip install "zenml[server]"
!zenml integration install sklearn -y
%pip install pyparsing==2.4.2  # required for Colab

import IPython

# automatically restart kernel
IPython.Application.instance().kernel.do_shutdown(restart=True)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 593.7/593.7 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.6/525.6 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.6/442.6 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.

{'status': 'ok', 'restart': True}

**Colab Note:** On Colab, you need an [ngrok account](https://dashboard.ngrok.com/signup) to view some of the visualizations later. Please set up an account, then set your user token below:

In [1]:
NGROK_TOKEN = "2SqcaHGFW0j4yB9afavkFqHR7Jy_22bRJ3oDDA8LfGKdkKMHo"  # TODO: set your ngrok token if you are working on Colab

In [2]:
from zenml.environment import Environment

if Environment.in_google_colab():  # Colab only setup

    # install and authenticate ngrok
    !pip install pyngrok
    !ngrok authtoken {NGROK_TOKEN}

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


As an ML practitioner, you are probably familiar with building ML models using Scikit-learn, PyTorch, TensorFlow, or similar. An **[ML Pipeline](https://docs.zenml.io/user-guide/starter-guide)** is simply an extension, including other steps you would typically do before or after building a model, like data acquisition, preprocessing, model deployment, or monitoring. The ML pipeline essentially defines a step-by-step procedure of your work as an ML practitioner. Defining ML pipelines explicitly in code is great because:
- We can easily rerun all of our work, not just the model, eliminating bugs and making our models easier to reproduce.
- Data and models can be versioned and tracked, so we can see at a glance which dataset a model was trained on and how it compares to other models.
- If the entire pipeline is coded up, we can automate many operational tasks, like retraining and redeploying models when the underlying problem or data changes or rolling out new and improved models with CI/CD workflows.

Having a clearly defined ML pipeline is essential for ML teams that aim to serve models on a large scale.

## ZenML Setup
Throughout this series, we will define our ML pipelines using [ZenML](https://github.com/zenml-io/zenml/). ZenML is an excellent tool for this task, as it is straightforward and intuitive to use and has [integrations](https://zenml.io/integrations) with most of the advanced MLOps tools we will want to use later. Make sure you have ZenML installed (via `pip install zenml`). Next, let's run some commands to make sure you start with a fresh ML stack.

In [3]:
!rm -rf .zen
!zenml init

Initializing the ZenML global configuration version to 0.83.1
⠋ Initializing ZenML repository at /content.
⠙ Initializing ZenML repository at /content.
⠹ Initializing ZenML repository at /content.
⠸ Initializing ZenML repository at /content.
⠴ Initializing ZenML repository at /content.
⠧ Initializing ZenML repository at /content.
⠇ Initializing ZenML repository at /content.
⠏ Initializing ZenML repository at /content.
⠹ Initializing ZenML repository at /content.
⠸ Initializing ZenML repository at /content.
⠼ Initializing ZenML repository at /content.
⠴ Initializing ZenML repository at /content.
⠦ Initializing ZenML repository at /content.
⠇ Initializing ZenML repository at /content.
⠏ Initializing ZenML repository at /content.
⠋ Initializing ZenML repository at /content.
⠙ Initializing ZenML repository at /content.
⠹ Initializing ZenML repository at /content.
⠸ Initializing ZenML repository at /content.
⠼ Initializing ZenML repository at /content.
⠴ Initializing ZenML repository at /co

## Example Experimentation ML Code
Let us get started with some simple exemplary ML code. In the following, we train a Scikit-learn SVC classifier to classify images of handwritten digits. We load the data, train a model on the training set, then test it on the test set.

In [9]:
import numpy as np
from sklearn.base import ClassifierMixin
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split


def train_test() -> None:
    """Train and test a Scikit-learn SVC classifier on digits"""
    digits = load_digits()
    data = digits.images.reshape((len(digits.images), -1))
    X_train, X_test, y_train, y_test = train_test_split(
        data, digits.target, test_size=0.2, shuffle=False
    )
    model = SVC(gamma=0.001)
    model.fit(X_train, y_train)
    test_acc = model.score(X_test, y_test)
    print(f"Test accuracy: {test_acc}")


train_test()

Test accuracy: 0.9583333333333334


## Turning experiments into ML pipelines with ZenML

In practice, your ML workflows will, of course, be much more complicated than that. You might have complex preprocessing that you do not want to redo every time you train a model, you will need to compare the performance of different models, deploy them in a production setting, and much more. Here ML pipelines come into play, allowing us to define our workflows in modular steps that we can then mix and match.

![Digits Pipeline](https://github.com/zenml-io/zenbytes/blob/main/_assets/1-1/digits_pipeline.png?raw=1)

We can identify three distinct steps in our example: data loading, model training, and model evaluation. Let us now define each of them as a ZenML **[Pipeline Step](https://docs.zenml.io/user-guide/starter-guide)** simply by moving each step to its own function and decorating them with ZenML's `@step` [Python decorator](https://realpython.com/primer-on-python-decorators/).

In [10]:
from typing import Tuple
import numpy as np
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.base import ClassifierMixin
from zenml.steps import step

@step
def importer() -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    """Load the digits dataset as numpy arrays."""
    digits = load_digits()
    data = digits.images.reshape((len(digits.images), -1))
    X_train, X_test, y_train, y_test = train_test_split(
        data, digits.target, test_size=0.2, shuffle=False
    )
    return X_train, X_test, y_train, y_test

@step
def svc_trainer(
    X_train: np.ndarray,
    y_train: np.ndarray,
) -> ClassifierMixin:
    """Train an sklearn SVC classifier."""
    model = SVC(gamma=0.001)
    model.fit(X_train, y_train)
    return model

@step
def evaluator(
    X_test: np.ndarray,
    y_test: np.ndarray,
    model: ClassifierMixin,
) -> float:
    """Calculate the test set accuracy of an sklearn model."""
    test_acc = model.score(X_test, y_test)
    print(f"Test accuracy: {test_acc}")
    return test_acc


Similarly, we can use ZenML's `@pipeline` decorator to connect all of our steps into an ML pipeline.

Note that the pipeline definition does not depend on the concrete step functions we defined above; it merely establishes a recipe for how data moves through the steps. This means we can replace steps as we wish, e.g., to run the same pipeline with different models to compare their performances.

In [11]:
from zenml.pipelines import pipeline


@pipeline
def digits_pipeline(importer, trainer, evaluator):
    """Links all the steps together in a pipeline"""
    X_train, X_test, y_train, y_test = importer()
    model = trainer(X_train=X_train, y_train=y_train)
    evaluator(X_test=X_test, y_test=y_test, model=model)

## Running ZenML Pipelines
Finally, we initialize our pipeline with concrete step functions and call the `run()` method to run it.

In [12]:
digits_svc_pipeline = digits_pipeline(
    importer=importer(), trainer=svc_trainer(), evaluator=evaluator()
)
digits_svc_pipeline.run(unlisted=True)

Running single step pipeline to execute step importer
Initiating a new run for the pipeline: importer.
Caching is disabled by default for importer.
Using user: default
Using stack: default
  orchestrator: default
  artifact_store: default
You can visualize your pipeline runs in the ZenML Dashboard. In order to try it locally, please run zenml login --local.
Step importer has started.
Step importer has finished in 11.291s.
Pipeline run has finished in 12.667s.
Running single step pipeline to execute step svc_trainer


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /usr/local/lib/python3.11/dist-packages/zenml/steps/utils.py:535 in run_as_single_step_pipeline  │
│                                                                                                  │
│   532 │                                                                                          │
│   533 │   try:                                                                                   │
│   534 │   │   validated_arguments = (                                                            │
│ ❱ 535 │   │   │   inspect.signature(__step.entrypoint)                                           │
│   536 │   │   │   .bind(*args, **kwargs)                                                         │
│   537 │   │   │   .arguments                                                                     │
│   538 │   │   )                                                                                  │
│                                                                                                  │
│ /usr/lib/python3.11/inspect.py:3195 in bind                                                      │
│                                                                                                  │
│   3192 │   │   and `kwargs` to the function's signature.  Raises `TypeError`                     │
│   3193 │   │   if the passed arguments can not be bound.                                         │
│   3194 │   │   """                                                                               │
│ ❱ 3195 │   │   return self._bind(args, kwargs)                                                   │
│   3196 │                                                                                         │
│   3197 │   def bind_partial(self, /, *args, **kwargs):                                           │
│   3198 │   │   """Get a BoundArguments object, that partially maps the                           │
│                                                                                                  │
│ /usr/lib/python3.11/inspect.py:3110 in _bind                                                     │
│                                                                                                  │
│   3107 │   │   │   │   │   │   else:                                                             │
│   3108 │   │   │   │   │   │   │   msg = 'missing a required argument: {arg!r}'                  │
│   3109 │   │   │   │   │   │   │   msg = msg.format(arg=param.name)                              │
│ ❱ 3110 │   │   │   │   │   │   │   raise TypeError(msg) from None                                │
│   3111 │   │   │   else:                                                                         │
│   3112 │   │   │   │   # We have a positional argument to process                                │
│   3113 │   │   │   │   try:                                                                      │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: missing a required argument: 'X_train'

The above exception was the direct cause of the following exception:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 0>:2                                                                              │
│                                                                                                  │
│   1 digits_svc_pipeline = digits_pipeline(                                                       │
│ ❱ 2 │   importer=importer(), trainer=svc_trainer(), evaluator=evaluator()                        │
│   3 )                                                                                            │
│   4 digits_svc_pipeline.run(unlisted=True)                                                       │
│   5                                    

And that's it, we just built and ran our first ML pipeline! Great job!

You can now visualize the pipeline run in ZenML's dashboard. To do so, run
`zenml up` to spin up a ZenML dashboard locally, log in with username `default`
and empty password, and navigate to the "Runs" tab in the "Pipelines" section.

In [ ]:
from zenml.environment import Environment

def start_zenml_dashboard(port=8237):
    if Environment.in_google_colab():
        from pyngrok import ngrok

        public_url = ngrok.connect(port)
        print(f"\x1b[31mIn Colab, use this URL instead: {public_url}!\x1b[0m")
        !zenml up --blocking --port {port}

    else:
        !zenml up --port {port}

start_zenml_dashboard()

INFO:pyngrok.ngrok:Opening tunnel named: http-8237-e806ece8-eb42-4fb2-b043-5cc9bcb61251


Opening tunnel named: http-8237-e806ece8-eb42-4fb2-b043-5cc9bcb61251


INFO:pyngrok.process.ngrok:t=2025-07-07T06:50:31+0000 lvl=info msg="no configuration paths supplied"


t=2025-07-07T06:50:31+0000 lvl=info msg="no configuration paths supplied"


INFO:pyngrok.process.ngrok:t=2025-07-07T06:50:31+0000 lvl=info msg="using configuration at default config path" path=/root/.config/ngrok/ngrok.yml


...

t=2025-07-07T06:50:32+0000 lvl=info msg=start pg=/api/tunnels id=9927cb9c7c6fb4f9


INFO:pyngrok.process.ngrok:t=2025-07-07T06:50:32+0000 lvl=info msg="started tunnel" obj=tunnels name=http-8237-e806ece8-eb42-4fb2-b043-5cc9bcb61251 addr=http://localhost:8237 url=https://1cee-34-145-225-225.ngrok-free.app


In Colab, use this URL instead: NgrokTunnel: "https://1cee-34-145-225-225.ngrok-free.app" -> "http://localhost:8237"!
t=2025-07-07T06:50:32+0000 lvl=info msg="started tunnel" obj=tunnels name=http-8237-e806ece8-eb42-4fb2-b043-5cc9bcb61251 addr=http://localhost:8237 url=https://1cee-34-145-225-225.ngrok-free.app


INFO:pyngrok.process.ngrok:t=2025-07-07T06:50:32+0000 lvl=info msg=end pg=/api/tunnels id=9927cb9c7c6fb4f9 status=201 dur=46.125039ms


t=2025-07-07T06:50:32+0000 lvl=info msg=end pg=/api/tunnels id=9927cb9c7c6fb4f9 status=201 dur=46.125039ms
The `zenml up` command is deprecated and will be removed in a future release. 
Please use the `zenml login --local` command instead.
Calling `zenml login --local`...
The local ZenML dashboard is about to deploy in a blocking process.
Deploying a local daemon ZenML server.
Not writing the global configuration to disk in a ZenML server environment.
Initializing the ZenML global configuration version to 0.83.1
Not writing the global configuration to disk in a ZenML server environment.
Starting ZenML Server as blocking process... press CTRL+C once to stop it.
INFO:     Started server process [2251]
INFO:     Waiting for application startup.
Not writing the global configuration to disk in a ZenML server environment.
Not writing the global configuration to disk in a ZenML server environment.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8237 (Pres

![Viewing Pipeline Runs in the ZenML Dashboard](https://github.com/zenml-io/zenbytes/blob/main/_assets/1-1/view_run_in_dashboard.gif?raw=1)

In the [next lesson](1-2_Artifact_Lineage.ipynb), you will see one of the best features of ML pipelines in action: automated artifact versioning and caching. See you there!